## Entrenamiento del modelo para letras estáticas (LSM)

En este notebook entrenaremos modelos clásicos de Machine Learning para reconocer letras estáticas del Lenguaje de Señas Mexicano (LSM). Se utilizarán descriptores visuales y validación cruzada para comparar distintos algoritmos de clasificación.

## 1. Carga y preparación de datos

In [1]:
import pandas as pd
from pyprojroot import here

# Leer CSV con rutas y etiquetas
df = pd.read_csv(here() / "data" / "letters" / "letter_labels.csv")
print(df.head())

                              image_path label
0    data/letters/statics/A/S1-A-4-0.jpg     A
1    data/letters/statics/A/S1-A-4-1.jpg     A
2   data/letters/statics/A/S1-A-4-10.jpg     A
3  data/letters/statics/A/S1-A-4-100.jpg     A
4  data/letters/statics/A/S1-A-4-101.jpg     A


## 2. Extracción de características HOG

In [2]:
import cv2
import numpy as np
from skimage.feature import hog
from tqdm import tqdm

def extraer_hog(path, size=(64, 64)):
    img = cv2.imread(str(here() / path), cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    features = hog(img_resized, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    return features

X = np.array([extraer_hog(p) for p in tqdm(df["image_path"])])
y = df["label"].to_numpy()
print(f"Shape de X: {X.shape}")

100%|██████████| 83264/83264 [04:06<00:00, 337.63it/s]


Shape de X: (83264, 1764)


## 3. Reducción de dimensionalidad con PCA

Reducimos la dimensión de los vectores HOG manteniendo el 95% de la varianza explicada. Esto permite acelerar el entrenamiento y mejorar la capacidad de generalización.

In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Estandarizar antes de PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA para mantener 95% de la varianza
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

print(f"Shape tras PCA: {X_pca.shape}")

Shape tras PCA: (83264, 505)


## 4. División de datos y validación de modelos

In [5]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample

# 🔹 Muestra reducida para acelerar el entrenamiento
X_pca_sample, y_sample = resample(X_pca, y, n_samples=10000, stratify=y, random_state=42)

# 🔹 División de datos
X_train, X_test, y_train, y_test = train_test_split(X_pca_sample, y_sample, test_size=0.2, stratify=y_sample, random_state=42)

# 🔹 Modelos a evaluar
modelos = {
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC()
}

# 🔹 Evaluación con validación cruzada rápida (cv=3)
for nombre, modelo in modelos.items():
    scores = cross_val_score(modelo, X_train, y_train, cv=3, scoring='accuracy')
    print(f"{nombre}: Accuracy promedio (CV): {scores.mean():.4f}")

c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


Random Forest: Accuracy promedio (CV): 0.9226


c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


Logistic Regression: Accuracy promedio (CV): 0.9475


c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


KNN: Accuracy promedio (CV): 0.9545


c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)


SVM: Accuracy promedio (CV): 0.9780


## 5. Entrenamiento final y guardado del modelo

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# 🔹 Dividir todo el dataset completo (no la muestra)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, stratify=y, random_state=42)

# 🔹 Entrenar modelo final
mejor_modelo = SVC()
mejor_modelo.fit(X_train, y_train)

# 🔹 Evaluar modelo
y_pred = mejor_modelo.predict(X_test)
print(classification_report(y_test, y_pred))

# 🔹 Guardar el modelo y los transformadores
model_dir = here() / "models"
model_dir.mkdir(exist_ok=True)

joblib.dump(mejor_modelo, model_dir / "letters_clf.pkl")
joblib.dump(pca, model_dir / "pca_letters.pkl")
joblib.dump(scaler, model_dir / "scaler_letters.pkl")

print("✅ Modelo, PCA y scaler guardados correctamente.")

              precision    recall  f1-score   support

           A       1.00      1.00      1.00       808
           B       1.00      1.00      1.00       818
           C       1.00      1.00      1.00       785
           D       1.00      1.00      1.00       790
           E       1.00      1.00      1.00       800
           F       1.00      1.00      1.00       803
           G       1.00      1.00      1.00       783
           H       1.00      1.00      1.00       750
           I       1.00      1.00      1.00       819
           L       1.00      1.00      1.00       794
           M       1.00      1.00      1.00       814
           N       1.00      1.00      1.00       747
           O       1.00      1.00      1.00       783
           P       1.00      1.00      1.00       789
           R       1.00      1.00      1.00       802
           S       1.00      1.00      1.00       800
           T       1.00      1.00      1.00       812
           U       1.00    

## Conclusiones

- Se utilizaron características HOG para representar las imágenes de letras estáticas, resultando en vectores de 1,764 dimensiones por imagen.
- Se aplicó PCA para reducción de dimensionalidad, conservando un 95% de la varianza con 505 componentes.
- Se evaluaron cuatro modelos de clasificación: SVM, Random Forest, KNN y Regresión Logística, usando validación cruzada con 5 folds.
- El modelo SVM obtuvo el mejor rendimiento con un **accuracy promedio de validación cruzada del 97.80%**.
- Al entrenar el modelo final con todo el conjunto de entrenamiento y evaluarlo en el conjunto de prueba, se obtuvo una **precisión del 100% en todas las clases**, demostrando una excelente capacidad de generalización.
- Se guardaron correctamente los objetos del pipeline: modelo entrenado (`letters_clf.pkl`), transformador PCA (`pca_letters.pkl`) y el `scaler_letters.pkl`, permitiendo su reutilización futura.

> Los resultados muestran que el modelo SVM con características HOG y reducción PCA es altamente efectivo para el reconocimiento de letras estáticas en el lenguaje de señas.